In [1]:
import pypylon.pylon as py
import numpy as np
import matplotlib.pyplot as plt
import traceback
import time
from datetime import datetime
import cv2
import subprocess as sp
import os

In [2]:
width = 1080
height = 1920
fps = 30

In [3]:
tlf = py.TlFactory.GetInstance()
devices = tlf.EnumerateDevices()
cam_count = len(devices)
cam_array = py.InstantCameraArray(cam_count)

for d in devices:
    print(d.GetModelName(), d.GetUserDefinedName())

for idx, cam in enumerate(cam_array):
    cam.Attach(tlf.CreateDevice(devices[idx]))

daA1920-30uc Wide
daA1920-30uc Narrow


In [4]:
cam_array.Open()

In [5]:
for idx, cam in enumerate(cam_array):
    camera_name = cam.DeviceInfo.GetUserDefinedName()
    print(f"set context {idx} for {camera_name} camera")
    cam.SetCameraContext(idx)

set context 0 for Wide camera
set context 1 for Narrow camera


In [6]:
def initialize_cam(cam):
    cam.UserSetSelector = 'Default'
    cam.UserSetLoad.Execute()

    cam.ExposureAuto = 'Off'
    cam.PixelFormat = 'YCbCr422_8'
    cam.ExposureTime = 30000
    cam.AcquisitionFrameRate = fps

In [7]:
for idx, cam in enumerate(cam_array):
    initialize_cam(cam)

In [8]:
class FFMPEG_VideoWriter:
    """ A class for FFMPEG-based video writing.

    A class to write videos using ffmpeg. ffmpeg will write in a large
    choice of formats.

    Parameters
    -----------

    filename
      Any filename like 'video.mp4' etc. but if you want to avoid
      complications it is recommended to use the generic extension
      '.avi' for all your videos.

    size
      Size (width,height) of the output video in pixels.

    fps
      Frames per second in the output video file.

    codec
      FFMPEG codec. It seems that in terms of quality the hierarchy is
      'rawvideo' = 'png' > 'mpeg4' > 'libx264'
      'png' manages the same lossless quality as 'rawvideo' but yields
      smaller files. Type ``ffmpeg -codecs`` in a terminal to get a list
      of accepted codecs.

      Note for default 'libx264': by default the pixel format yuv420p
      is used. If the video dimensions are not both even (e.g. 720x405)
      another pixel format is used, and this can cause problem in some
      video readers.

      Experimentally found best options 
        libx264         - quality - very good     speed - ~30fps achieved       size - 16.74 GB/h
        libx265         - quality - very good     speed - ~15fps achieved       size - 1.396 GB/h
        mjpeg(-q:v=25)  - quality - good          speed - ~30fps achieved       size - 3.66 GB/h
        mpeg(-q:v=11)   - quality - very good     speed - ~30fps achieved       size - 1.624 GB/h
 -
    audiofile
      Optional: The name of an audio file that will be incorporated
      to the video.

    preset
      Sets the time that FFMPEG will take to compress the video. The slower,
      the better the compression rate. Possibilities are: ultrafast,superfast,
      veryfast, faster, fast, medium (default), slow, slower, veryslow,
      placebo. 

      This and affects only for the libx264, libx265 libxvid etc. ('-crf' also affect these types)
      for mjpeg, mpeg4 etc. use -q:v factor

    bitrate
      Only relevant for codecs which accept a bitrate. "5000k" offers
      nice results in general.

    withmask
      Boolean. Set to ``True`` if there is a mask in the video to be
      encoded.

    """

    def __init__(self, filename, size, fps, codec="libx264", audiofile=None,
                 preset="medium", bitrate=None, pixfmt="rgba", quality = '11',crf = '20',
                 logfile=None, threads=None, ffmpeg_params=None):

        if logfile is None:
            logfile = sp.PIPE

        self.filename = filename
        self.codec = codec
        self.ext = self.filename.split(".")[-1]

        # order is important
        cmd = [
            "ffmpeg",
            '-y',
            '-loglevel', 'error' if logfile == sp.PIPE else 'info',
            '-f', 'rawvideo',
            '-vcodec', 'rawvideo',
            '-s', '%dx%d' % (size[1], size[0]),
            '-pix_fmt', pixfmt,
            '-r', '%.02f' % fps,
            '-i', '-', '-an',
        ]
        cmd.extend([
            '-vcodec', codec,
            '-q:v', quality,
            '-crf', crf,
            '-preset', preset,
        ])
        if ffmpeg_params is not None:
            cmd.extend(ffmpeg_params)
        if bitrate is not None:
            cmd.extend([
                '-b', bitrate
            ])
        if threads is not None:
            cmd.extend(["-threads", str(threads)])

        if ((codec == 'libx264') and
                (size[0] % 2 == 0) and
                (size[1] % 2 == 0)):
            cmd.extend([
                '-pix_fmt', 'yuv420p'
            ])
        cmd.extend([
            filename
        ])

        popen_params = {"stdout": sp.DEVNULL,
                        "stderr": logfile,
                        "stdin": sp.PIPE,
                        "shell":True}

        # This was added so that no extra unwanted window opens on windows
        # when the child process is created
        if os.name == "nt":
            popen_params["creationflags"] = 0x08000000  # CREATE_NO_WINDOW

        self.proc = sp.Popen(cmd, **popen_params)


    def write_frame(self, img_array):
        """ Writes one frame in the file."""
        try:
               self.proc.stdin.write(img_array.tobytes())
        except IOError as err:
            _, ffmpeg_error = self.proc.communicate()
            error = (str(err) + ("\n\nMoviePy error: FFMPEG encountered "
                                 "the following error while writing file %s:"
                                 "\n\n %s" % (self.filename, str(ffmpeg_error))))

            if b"Unknown encoder" in ffmpeg_error:

                error = error+("\n\nThe video export "
                  "failed because FFMPEG didn't find the specified "
                  "codec for video encoding (%s). Please install "
                  "this codec or change the codec when calling "
                  "write_videofile. For instance:\n"
                  "  >>> clip.write_videofile('myvid.webm', codec='libvpx')")%(self.codec)

            elif b"incorrect codec parameters ?" in ffmpeg_error:

                 error = error+("\n\nThe video export "
                  "failed, possibly because the codec specified for "
                  "the video (%s) is not compatible with the given "
                  "extension (%s). Please specify a valid 'codec' "
                  "argument in write_videofile. This would be 'libx264' "
                  "or 'mpeg4' for mp4, 'libtheora' for ogv, 'libvpx for webm. "
                  "Another possible reason is that the audio codec was not "
                  "compatible with the video codec. For instance the video "
                  "extensions 'ogv' and 'webm' only allow 'libvorbis' (default) as a"
                  "video codec."
                  )%(self.codec, self.ext)

            elif  b"encoder setup failed" in ffmpeg_error:

                error = error+("\n\nThe video export "
                  "failed, possibly because the bitrate you specified "
                  "was too high or too low for the video codec.")

            elif b"Invalid encoder type" in ffmpeg_error:

                error = error + ("\n\nThe video export failed because the codec "
                  "or file extension you provided is not a video")


            raise IOError(error)

    def close(self):
        if self.proc:
            self.proc.stdin.close()
            if self.proc.stderr is not None:
                self.proc.stderr.close()
            self.proc.wait()

        self.proc = None

    # Support the Context Manager protocol, to ensure that resources are cleaned up.

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.close()


In [9]:
def draw_time_date(frame,cam_id):

    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (1150, 50)
    fontScale = 1
    color = (255, 255, 0)
    thickness = 2

    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

    frame = cv2.putText(frame, dt_string, org, font, fontScale, color, thickness, cv2.LINE_AA)

    # Attempt to display using cv2 (doesn't work)
    # if (cam_id == 0):
    #     cv2.namedWindow("1")
    #     cv2.imshow("1", frame)
    #     cv2.waitKey(1)
    # else:
    #     cv2.namedWindow("2")
    #     cv2.imshow("2", frame)
    #     cv2.waitKey(1)
        
    return frame

In [10]:
def save_video(frame,cam_id):
    # frame = np.frombuffer(frame, dtype=np.uint8).reshape(width, height, -1)
    # frame = cv2.cvtColor(cv2.resize(frame, (height,width)), cv2.COLOR_RGB2BGR)
    frame = draw_time_date(frame,cam_id)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2YUV_I420)

    writer[cam_id].write_frame(frame)
    # output_video.write(frame)
    return

In [11]:
converter = py.ImageFormatConverter()
converter.OutputPixelFormat = py.PixelType_RGB8packed
converter.OutputBitAlignment = "MsbAligned"

In [12]:
class ImageHandler (py.ImageEventHandler):
    def __init__(self, *args):
        super().__init__(*args)
        self.time_old = time.time()
        self.time_new = 0
    
    def OnImageGrabbed(self, camera, grabResult):
        try:
            if grabResult.GrabSucceeded():
                
                if (~converter.ImageHasDestinationFormat(grabResult)):
                    grabResult = converter.Convert(grabResult)
                    
                img = grabResult.Array
                cam_id = camera.CameraContext
                self.time_new = time.time()
                rate = 1/(self.time_new-self.time_old)
                if (cam_id == 0):
                    print("                     ",rate,cam_id)
                else:
                    print(rate, cam_id)
                self.time_old = self.time_new
                # img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB )
                # cv2.namedWindow("Input")
                # cv2.imshow("Input", img)
                # cv2.waitKey(1)
                save_video(img,cam_id)
            else:
                raise RuntimeError("Grab failed")
        except Exception as e:
            traceback.print_exc()

In [13]:
def BackgroundLoop(cam_array):

    handler_array = []
    for i in range (cam_count):
        handler_array.append(ImageHandler())

    for cam in cam_array:
        cam.RegisterImageEventHandler(handler_array[i], py.RegistrationMode_ReplaceAll, py.Cleanup_None)
        


    global writer
    writer = []
    for i in range(cam_count):
        writer.append(FFMPEG_VideoWriter("output_"+str(i)+".mp4",(width, height), fps=fps, pixfmt="yuv420p", codec="mpeg4", quality='11', preset= 'ultrafast'))
    # writer_1 = FFMPEG_VideoWriter("output_1.mp4",(cam.Height.Value, cam.Width.Value), fps=fps, pixfmt="yuv420p", codec="mpeg4", quality='11', preset= 'ultrafast')
    # writer_2 = FFMPEG_VideoWriter("output_2.mp4",(cam.Height.Value, cam.Width.Value), fps=fps, pixfmt="yuv420p", codec="mpeg4", quality='11', preset= 'ultrafast') 

    # cam.StartGrabbing(py.GrabStrategy_LatestImages, py.GrabLoop_ProvidedByInstantCamera)
    cam_array.StartGrabbing(py.GrabStrategy_LatestImages, py.GrabLoop_ProvidedByInstantCamera)

    try:
        while cam_array.IsGrabbing():
            pass
    except KeyboardInterrupt:
        pass

    cam_array.StopGrabbing()
    for i in range (cam_count):
        cam_array[i].DeregisterImageEventHandler(handler_array[i])
        writer[i].close()
    cam_array.Close()
    cv2.destroyAllWindows()
    return

In [14]:
BackgroundLoop(cam_array)

                     7.198853484600136 7.411714395778774 0
 1
                      15.915246262426956 0
111.36108751062022 1
14.119906143430882                       13.191460453647675 0
1
27.533784537821745 1
                      167.17700984495198 0
16.954697776322543                      15.880296834772073 0
 1
20.884431918898194                       18.91942460970802 0
1
                      20.459819904196056 0
125.48028480823311 1
                      21.33007862162959323.869654045994412  1
0
                      27.856728233942366 0
200.482959705559 1
                      23.872642932354363 0
250.60070502479536 1
                      30.386901398246756 0
200.50212725273676 1
                      29.488550638028613 0
                      25.068158454660644 0
77.03603570510231 1
                      47.7858111264284 0
83.55520140244631 1
                      41.76969576258527 0
77.14371896266323 1
                      31.3344489600765 0
100.28701910432059 1
          

Traceback (most recent call last):
  File "C:\Users\Samare\AppData\Local\Temp/ipykernel_1568/1730212487.py", line 17, in OnImageGrabbed
    rate = 1/(self.time_new-self.time_old)
ZeroDivisionError: float division by zero
Traceback (most recent call last):
  File "C:\Users\Samare\AppData\Local\Temp/ipykernel_1568/1730212487.py", line 17, in OnImageGrabbed
    rate = 1/(self.time_new-self.time_old)
ZeroDivisionError: float division by zero


 32.00781440781441 0
                      19.995156507727657 1
22.713780536015033 0
46.72902693910292 1
                      64.0107439908432 0
29.670871032321504 1
                      26.46666035652311 0
91.15872291408576 1


Traceback (most recent call last):
  File "C:\Users\Samare\AppData\Local\Temp/ipykernel_1568/1730212487.py", line 17, in OnImageGrabbed
    rate = 1/(self.time_new-self.time_old)
ZeroDivisionError: float division by zero


25.7096867127209                       21.79538557472459 0
1
25.06905743828821 1
                      200.42547904620824 0
24.45786659358217 1
                      306.7805734347572 0
64.01172089616018 1
                      18.289142862126255 0
167.06380944794074 1
                     25.423414030961705  1
42.17669891197232 0
26.547740061142722 1
                      200.482959705559 0
25.710632298403162                      24.014382393019503 0
 1
32.00781440781441 1                     
 32.00781440781441 0
24.219472337870066                       21.61009840795507 0
1
27.575600583818755 1
                      212.56355159132374 0
24.681958878152695 1
                      77.12385995881141 0
47.06302667160377 1
46.21262436509073                      46.21262436509073 0
 1
                      32.00830293502648 0
32.00732589551441                       25.698974933980356 0
1


...

                     33.7501830617582 1
 33.7501830617582 0
                      20.76624566163474 0
43.5975676939868 1
                      77.12244184977476 0
40.843134390854296 1
                      110.66181204158092 0
32.00757014980044 1
                      59.83485976775371 0
25.077750938703275                      1
 22.797484522858337 0
                      27.156563007853727 0
200.33931983186855 1
                     22.78584272714926  1
25.06905743828821 0
                      27.255739601135897 0
125.36027258054875 1
                      87.30130713512614 0
64.0107439908432 1
                      64.01758295430264 0
64.01367479625164 1
                      32.42901544789621 0
48.56823261038224 1
                      19.93916949917995821.66289117174628 1
 0
23.879710549239082                       22.296014756616824 0
1
29.692505911169633                      1
 26.55009273501839 0
37.13153561500735 1
                      60.339279548854876 0
                   

Traceback (most recent call last):
  File "C:\Users\Samare\AppData\Local\Temp/ipykernel_1568/1730212487.py", line 17, in OnImageGrabbed
    rate = 1/(self.time_new-self.time_old)
ZeroDivisionError: float division by zero


27.24210854485464 1
                      200.65559967468783 0
                      28.37977698386922 0
62.66796157121726 1
                      135.93595851563765 0
64.01758295430264 1
                      48.08658167476841 0
64.00976711534352 1
                      64.01758295430264 0
29.981586321267226 1
                      41.77801683350764 0
32.34374108375296                       28.648834731291494 0
1
                      29.050047789890705 0
66.84575909221304 1
                      37.13844001522973 0
111.40249667994688 1
                      49.37670257225263 0
63.976571079926785 1
                      32.47848475697106 0
83.54854388271383 1
                      52.61228534514118 0
64.00976711534352 1
                      76.85816902440813 0
                      29.059910069076373 0
45.58431508933617 1
                      58.98498059290094 0
201.8724551186408 1
32.00634891564794 1
                      35.755848052922325 0
200.35845992165855 1


Traceback (most recent call last):
  File "C:\Users\Samare\AppData\Local\Temp/ipykernel_1568/1730212487.py", line 17, in OnImageGrabbed
    rate = 1/(self.time_new-self.time_old)
ZeroDivisionError: float division by zero


                      21.7990104361565 0
38.559802894073954 1
                      52.78310660307313 0
111.38178824654115 1
21.335069585741028 1
                      250.66061076913883 0
29.490001968670022 1
                      125.28912387609404 0
25.712366052818716 1
                      77.12385995881141 0
35.81049306296691 1
                      45.57292334437985 0
66.8468244481632 1
                      35.80988158153116 0
111.39657919898013 1
                      40.10656059055833 0
91.15872291408576 1
                      37.13679587753006 0
200.3776036690235 1
                      40.11193037823364 0
111.40841478963026 1
                      25.05049153696382 0
47.80378390699795 1
                      83.56352479429403 0
                     25.066210877970942 28.648247693074783 0
 1
27.852103697407564 1
                      250.48097939683487 0
34.57480360396007 1
                      52.78111393551959 0
77.13520670884213 1
                      32.34399049954503

In [15]:
cam_array.Close()

In [15]:
cam_array.StopGrabbing()